In [1]:
import mpilot
import arcpy
import inspect
import pandas as pd
import geopandas as gpd
import os
import shutil
from shapely import wkt
from pyproj import CRS
from arcpy.sa import *

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

In [2]:
arcpy.ResetEnvironments()
gdb_name = "Buildings.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [5]:
path = os.path.join(THIS_FOLDER, "in_data/SBC_Buildings.shp")
arcpy.CopyFeatures_management(path, "SBC_Buildings")
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
arcpy.Project_management("SBC_Buildings", "SBC_Buildings_rpj", crs)

path = os.path.join(THIS_FOLDER, "out_data/ru/Reporting_Units_APN.shp")
arcpy.CopyFeatures_management(path, "Reporting_Units")
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
arcpy.Project_management("Reporting_Units", "Reporting_Units_rpj", crs)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Buildings.gdb\\Reporting_Units_rpj'>

In [6]:
arcpy.management.MultipartToSinglepart("SBC_Buildings_rpj", "SBC_Buildings_rpj_SP")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Buildings.gdb\\SBC_Buildings_rpj_SP'>

In [7]:
arcpy.Intersect_analysis(["Reporting_Units_rpj", "SBC_Buildings_rpj_SP"], "Building_Intersect")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Buildings.gdb\\Building_Intersect'>

In [10]:
dir_path = os.path.join(THIS_FOLDER, "out_data/building")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("Building_Intersect", os.path.join(THIS_FOLDER, "out_data/building"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\building'>

In [8]:
# HOUSING DENSITY

# arcpy.management.FeatureToPoint("SBC_Buildings_rpj_SP", "SBC_Buildings_Pts")  

# pdensOut = PointDensity("SBC_Buildings_Pts", "NONE", 200, NbrCircle(1000, "MAP"))
# out_raster = arcpy.ia.Int(pdensOut)
# arcpy.RasterToPolygon_conversion(out_raster, "point_density_1000ft", "NO_SIMPLIFY", "VALUE")

# pdensOut = PointDensity("SBC_Buildings_Pts", "NONE", 200, NbrCircle(2640, "MAP"))
# out_raster = arcpy.ia.Int(pdensOut)
# arcpy.RasterToPolygon_conversion(out_raster, "point_density_0p5mi", "NO_SIMPLIFY", "VALUE")

# pdensOut = PointDensity("SBC_Buildings_Pts", "NONE", 200, NbrCircle(15840, "MAP"))
# out_raster = arcpy.ia.Int(pdensOut)
# arcpy.RasterToPolygon_conversion(out_raster, "point_density_3mi", "NO_SIMPLIFY", "VALUE")

# arcpy.ClearWorkspaceCache_management()
# del pdensOut
# del out_raster

# dir_path = os.path.join(THIS_FOLDER, "out_data/dense_dev")
# shutil.rmtree(dir_path)
# os.mkdir(dir_path)


# arcpy.conversion.FeatureClassToShapefile("point_density_1000ft", 
#                                         os.path.join(THIS_FOLDER, "out_data/dense_dev"))

# arcpy.conversion.FeatureClassToShapefile("point_density_0p5mi", 
#                                         os.path.join(THIS_FOLDER, "out_data/dense_dev"))

# arcpy.conversion.FeatureClassToShapefile("point_density_3mi", 
#                                         os.path.join(THIS_FOLDER, "out_data/dense_dev"))

In [9]:
gdf = gpd.read_file(os.path.join(THIS_FOLDER, "in_data/SBC_Parcels.shp"))

Land Value, Land Use Type, Year Built, Acreage, Land Value, Sq Footage, Bathrooms, Ag Presence

In [10]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 132479 entries, 0 to 132478
Data columns (total 89 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   apn         132479 non-null  object  
 1   layer       132479 non-null  object  
 2   situs1      122560 non-null  object  
 3   situs2      122558 non-null  object  
 4   acreage     132479 non-null  float64 
 5   landuse     131568 non-null  object  
 6   usecode     131754 non-null  object  
 7   tra         131754 non-null  object  
 8   docnum      118501 non-null  object  
 9   docdate     132479 non-null  object  
 10  pcttransf   132479 non-null  float64 
 11  valreason   131537 non-null  object  
 12  nontaxcode  9101 non-null    object  
 13  sbeno       420 non-null     object  
 14  agpres      2174 non-null    object  
 15  landvalue   132479 non-null  int64   
 16  strimpr     132479 non-null  int64   
 17  tradefix    132479 non-null  int64   
 18  livimpr     1324

In [15]:
print(gdf['landuse'].unique())

['SINGLE FAMILY RESIDENCE' 'PASTURE OF GRAZING, DRY'
 'TRUCK CROPS-IRRIGATED' 'MISCELLANEOUS' 'RECREATION'
 'FIELD CROPS-IRRIGATED' 'UTILITY,WATER COMPANY' 'RIVERS AND LAKES'
 'IRRIGATED FARMS, MISC' 'RECREATIONAL OPEN (MISC)'
 'RANCHO ESTATES (RURAL HOME SITES)' 'COMMERCIAL (MISC)' 'VACANT' None
 'LIGHT MANUFACTURING' 'SCHOOLS' 'APARTMENTS, 5 OR MORE UNITS'
 'CHURCHES, RECTORY' 'WAREHOUSING' 'OTHER FOOD PROCESSING, BAKERIES'
 'INDUSTRIAL, MISC' 'AUTO SALES, REPAIR, STORAGE, CAR WASH, ETC'
 'INDUSTRIAL CONDOS,PUDS' 'FIELD CROPS, DRY' 'CONDOS,COMMUNITY APT PROJS'
 'ORCHARDS, IRRIGATED' 'BEACHES, SAND DUNES' 'VINEYARDS'
 'DRY FARMS (MISC)' 'RESIDENTIAL INCOME, 2-4 UNITS'
 'OFFICE BUILDINGS, SINGLE STORY' 'RIGHTS OF WAY,SEWER,LAND FILLS,ETC'
 'HORSES' 'PARKS' 'FLOWERS' 'NURSERIES,GREENHOUSES' 'WATER RIGHTS,PUMPS'
 'CAMPS, CABINS' 'OFFICE BUILDINGS, MULTI-STORY' 'HOTELS'
 'MOBILE HOME PARKS' 'SERVICE STATIONS' 'BANKS, S&LS'
 'SHOPPING CENTERS (NEIGHBORHOOD)' 'HIGHWAYS AND STREETS'
 'PETROL

In [38]:
field_names = [f.name for f in arcpy.ListFields("point_density_1000ft")]
print(field_names)

['OBJECTID', 'Shape', 'Id', 'gridcode', 'Shape_Length', 'Shape_Area']


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 132479 entries, 0 to 132478
Data columns (total 89 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   apn         132479 non-null  object  
 1   layer       132479 non-null  object  
 2   situs1      122560 non-null  object  
 3   situs2      122558 non-null  object  
 4   acreage     132479 non-null  float64 
 5   landuse     131568 non-null  object  
 6   usecode     131754 non-null  object  
 7   tra         131754 non-null  object  
 8   docnum      118501 non-null  object  
 9   docdate     132479 non-null  object  
 10  pcttransf   132479 non-null  float64 
 11  valreason   131537 non-null  object  
 12  nontaxcode  9101 non-null    object  
 13  sbeno       420 non-null     object  
 14  agpres      2174 non-null    object  
 15  landvalue   132479 non-null  int64   
 16  strimpr     132479 non-null  int64   
 17  tradefix    132479 non-null  int64   
 18  livimpr     1324

In [19]:
gdf["int_yr_blt"] = pd.to_numeric(gdf['yearbuilt'], downcast='integer')
gdf = gdf[gdf["int_yr_blt"] > 1900]

In [20]:
gdf.to_file(driver = 'ESRI Shapefile', filename= "out_data/final/Year_Built2.shp")

In [21]:
arcpy.ResetEnvironments()
gdb_name = "Year_Built.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
# arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [25]:
path = os.path.join(THIS_FOLDER, "out_data/final/Year_Built2.shp")
arcpy.CopyFeatures_management(path, "Year_Built")
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
arcpy.Project_management("Year_Built", "Year_Built_rpj", crs)

path = os.path.join(THIS_FOLDER, "out_data/final/Final_EEMS_Output.shp")
arcpy.CopyFeatures_management(path, "Final_EEMS_Output")
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
arcpy.Project_management("Final_EEMS_Output", "Final_EEMS_Output_rpj", crs)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Year_Built.gdb\\Final_EEMS_Output_rpj'>

In [26]:
field_names = [f.name for f in arcpy.ListFields("Year_Built_rpj")]
print(field_names)

['OBJECTID', 'Shape', 'apn', 'layer', 'situs1', 'situs2', 'acreage', 'landuse', 'usecode', 'tra', 'docnum', 'docdate', 'pcttransf', 'valreason', 'nontaxcode', 'sbeno', 'agpres', 'landvalue', 'strimpr', 'tradefix', 'livimpr', 'perpropdec', 'perspropun', 'mobilehome', 'exemptions', 'exempcode', 'homeowex', 'netsecval', 'net_impr', 'net_pers', 'net_unx', 'net_av', 'mailx1', 'mailx2', 'mnumber', 'mfrac', 'mdir', 'mstreet', 'mstrsuffix', 'munittype', 'munitnumb', 'pobox', 'mcity', 'mzip', 'mzipext', 'mstate', 'country', 'snum', 'sfra', 'sdir', 'sstreet', 'sstreetsuf', 'sunittype', 'sunitnumb', 'scity', 'szip', 'szipext', 'm_address1', 'm_address2', 'asmrollid', 'apn9', 'propid', 'sqfootage', 'yearbuilt', 'bedrooms', 'bathrooms', 'taxbill', 'permit', 'bkpglnk', 'gen_inquir', 'web_link', 'maptype', 'recmapnum', 'tractname', 'recmapbook', 'recmapdate', 'blocksecti', 'lotnum', 'unitnum', 'cbiinputs_', 'areaunit', 'recdatetim', 'recuserid', 'recuserlog', 'section', 'township', 'range', 'shape_le

In [27]:
arcpy.analysis.SummarizeWithin("Final_EEMS_Output_rpj", "Year_Built_rpj", 
                               "EEMS_WYRBUILT", "KEEP_ALL", "int_yr_blt Mean", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Year_Built.gdb\EEMS_WYRBUILT
1,
